In [16]:
import io
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

In [17]:
def is_resp_content_type_json(response) -> bool:
    return 'application/json' in response.headers['Content-Type']

In [18]:
def ades_GetPointEau(numéro_département: int=None, nb_réseau: int=None):
    """Récupérer et formater les données"""
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEau'
    post_params = {
        'type': '1',
        'filter': f'departement={numéro_département}&nbReseau={nb_réseau}',
    }
    response_GetPointEau = requests.post(url=url, data=post_params)
    if not is_resp_content_type_json(response_GetPointEau):
        return None
    response_payload = response_GetPointEau.json()
    point_eau_liste = response_payload['pointEaux']
    point_eau_liste_formatée = []
    for p in point_eau_liste:
        dic = {}
        dic['coordinates0'] = p['Coordinates'][0]
        dic['coordinates1'] = p['Coordinates'][1]
        dic['coord_com'] = p['Properties']['coord_com']
        dic['coord_ref0'] = p['Properties']['coord_ref'][0]
        dic['coord_ref1'] = p['Properties']['coord_ref'][1]
        dic['epsg_ref'] = p['Properties']['epsg_ref']
        dic['exist_mesures'] = p['Properties']['exist_mesures']
        if ' ' in p['Properties']['label']:
            dic['Identifiant BSS'] = p['Properties']['label'].split(' ')[0]
            dic['Ancien code national BSS'] = p['Properties']['label'].split(' ')[1].replace('(', '').replace(')', '')
        else:
            dic['Identifiant BSS'] = p['Properties']['label']
            dic['Ancien code national BSS'] = 'N/D'
        dic['position_captage'] = p['Properties']['position_captage']
        dic['type_pt_eau'] = p['Properties']['type_pt_eau']
        point_eau_liste_formatée.append(dic)
        return point_eau_liste_formatée

In [19]:
def ades_GetPointEauIPSDataCSV(numéro_département=None, nombre_points=None):
    """Retourne le contenu d'un fichier CSV sous forme d'octets"""
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV'
    query_params = {
        'filters': f'departement={numéro_département}',
        'nombre_points': nombre_points,
        'typeRecherche': 1,
    }
    response_GetPointEauIPSDataCSV = requests.get(url=url, params=query_params)
    content_as_bytes = response_GetPointEauIPSDataCSV.content
    return content_as_bytes

In [20]:
def ades_FicheMesureTabDonnees(row):
    url = 'https://ades.eaufrance.fr/Fiche/MesureTabDonnees'
    post_params = {
        'CodeBSS': row['Ancien code national BSS'],
        'DateDebut': 0,
        'DateFin': row['Date de fin'],
        'length': row['Nb de mesures'],
    }
    response_FicheMesureTabDonnees = requests.post(url, data=post_params)
    if not is_resp_content_type_json(response_FicheMesureTabDonnees):
        return None
    response_payload = response_FicheMesureTabDonnees.json()
    response_data = response_payload['data']
    noms_colonnes = ['Date', 'Cote piézo. (mNGF)', "Mode d'obtention", 'Nature de la mesure', 'Qualification', 'Statut', 'Producteur']
    nature_mesure_par_défaut = 'Naturel'
    df_FicheMesureTabDonnees = pd.DataFrame(response_data, columns=noms_colonnes)
    df_FicheMesureTabDonnees.loc[:, 'Cote piézo. (mNGF)'] = df_FicheMesureTabDonnees['Cote piézo. (mNGF)'].str.replace(',', '.')
    df_FicheMesureTabDonnees.loc[:, 'Cote piézo. (mNGF)'] = df_FicheMesureTabDonnees['Cote piézo. (mNGF)'].astype(dtype=np.float64)
    df_FicheMesureTabDonnees.loc[df_FicheMesureTabDonnees['Nature de la mesure'] == '',
                                 'Nature de la mesure'] = nature_mesure_par_défaut
    df_FicheMesureTabDonnees['Ancien code national BSS'] = row['Ancien code national BSS']
    df_FicheMesureTabDonnees['Identifiant BSS'] = row['Identifiant BSS']
    return df_FicheMesureTabDonnees

In [21]:
def get_mesure_tab_donnees_for_every_point_eau(df_point_eau: pd.DataFrame) -> pd.DataFrame:
    points_eau_liste = df_point_eau.to_dict(orient='records')
    points_eau_liste_row0 = points_eau_liste[0]
    df_fiche_mesure_row0 = ades_FicheMesureTabDonnees(points_eau_liste_row0)
    if df_fiche_mesure_row0 is None:
        return None
    df_full = pd.merge(left=df_fiche_mesure_row0,
                        right=pd.DataFrame([points_eau_liste_row0]),
                        on=['Ancien code national BSS', 'Identifiant BSS'],
                        how='outer')
    for points_eau_liste_row in tqdm(points_eau_liste[1:]):
        df_fiche_mesure_row = ades_FicheMesureTabDonnees(points_eau_liste_row)
        if df_fiche_mesure_row is None:
            continue
        df_tmp = pd.merge(left=df_fiche_mesure_row,
                        right=pd.DataFrame([points_eau_liste_row]),
                        on=['Ancien code national BSS', 'Identifiant BSS'],
                        how='outer')
        df_full = pd.concat([df_full, df_tmp])
    return df_full

In [22]:
numéro_département = 66

point_eau_liste = ades_GetPointEau(numéro_département, nb_réseau=0)

point_eau_ips_data_csv_bytes = ades_GetPointEauIPSDataCSV(numéro_département, 
                                                          nombre_points=len(point_eau_liste))

df_point_eau = pd.merge(left=pd.DataFrame(point_eau_liste),
                        right=pd.read_csv(io.BytesIO(point_eau_ips_data_csv_bytes), sep=';'),
                        on=['Ancien code national BSS', 'Identifiant BSS'],
                        how='outer')

df_ades_full = get_mesure_tab_donnees_for_every_point_eau(df_point_eau)

100%|██████████| 65/65 [00:34<00:00,  1.88it/s]


,Date,Cote piézo. (mNGF),Mode d'obtention,Nature de la mesure,Qualification,Statut,Producteur,Ancien code national BSS,Identifiant BSS,coordinates0,...,exist_mesures,position_captage,type_pt_eau,Type de suivi,Commune,Département,Nb de mesures,Date de début,Date de fin,Nb de réseaux
0,16/08/2023,1508.32,Valeur mesurée,Naturel,Non qualifié,Donnée brute,BRGM,10947X0084/F,BSS002MSAL,2.033499,...,None,None,1,Piézomètre,Font-Romeu-Odeillo-Via (66124),Pyrénées-Orientales (66),4068,27/06/2012,16/08/2023,6
1,15/08/2023,1508.32,Valeur mesurée,Naturel,Non qualifié,Donnée brute,BRGM,10947X0084/F,BSS002MSAL,2.033499,...,None,None,1,Piézomètre,Font-Romeu-Odeillo-Via (66124),Pyrénées-Orientales (66),4068,27/06/2012,16/08/2023,6
2,14/08/2023,1508.32,Valeur mesurée,Naturel,Non qualifié,Donnée brute,BRGM,10947X0084/F,BSS002MSAL,2.033499,...,None,None,1,Piézomètre,Font-Romeu-Odeillo-Via (66124),Pyrénées-Orientales (66),4068,27/06/2012,16/08/2023,6
3,13/08/2023,1508.31,Valeur mesurée,Naturel,Non qualifié,Donnée brute,BRGM,10947X0084/F,BSS002MSAL,2.033499,...,None,None,1,Piézomètre,Font-Romeu-Odeillo-Via (66124),Pyrénées-Orientales (66),4068,27/06/2012,16/08/2023,6
4,12/08/2023,1508.3,Valeur mesurée,Naturel,Non qualifié,Donnée brute,BRGM,10947X0084/F,BSS002MSAL,2.033499,...,None,None,1,Piézomètre,Font-Romeu-Odeillo-Via (66124),Pyrénées-Orientales (66),4068,27/06/2012,16/08/2023,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5402,21/10/2008,630.05,Valeur mesurée,Naturel,Correcte,Donnée contrôlée niveau 1,BRGM,11001X0005/CECILE,BSS002MVXJ,NaN,...,NaN,NaN,NaN,Piézomètre et qualitomètre,Le Tech (66206),Pyrénées-Orientales (66),5404,20/10/2008,16/08/2023,7
5403,20/10/2008,630.05,Valeur mesurée,Naturel,Correcte,Donnée contrôlée niveau 1,BRGM,11001X0005/CECILE,BSS002MVXJ,NaN,...,NaN,NaN,NaN,Piézomètre et qualitomètre,Le Tech (66206),Pyrénées-Orientales (66),5404,20/10/2008,16/08/2023,7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/D,BSS003FATM,NaN,...,NaN,NaN,NaN,Piézomètre,Trouillas (66217),Pyrénées-Orientales (66),1634,11/12/2018,13/08/2023,1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/D,BSS003FAXY,NaN,...,NaN,NaN,NaN,Piézomètre,Rivesaltes (66164),Pyrénées-Orientales (66),1620,11/12/2018,13/08/2023,1


In [25]:
df_ades_full.to_csv(path_or_buf=f'df_ades_full_dep_{numéro_département}.csv', index=False)